# Lab 11 — Training Deep Networks

> **강의 시간:** 약 2시간
> **주제:** 딥 신경망 학습: 옵티마이저, 정규화, 배치 정규화, 학습률 스케줄링

---

## 학습 목표

| # | 목표 | 예상 시간 |
|---|---|---|
| 1 | 옵티마이저 비교: SGD, Momentum, RMSProp, Adam | 30분 |
| 2 | 정규화 기법: L1/L2, Dropout | 25분 |
| 3 | 배치 정규화 & 레이어 정규화 | 25분 |
| 4 | 학습률 스케줄링 | 15분 |
| 5 | Exercise | 25분 |

---

**데이터셋:**
- 시각화: 합성 2D 데이터 (최적화 궤적, 과적합 비교)
- 분류: Digits (sklearn) — 64개 특성, 10개 클래스 (손글씨 숫자 0–9)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

_fp = '/System/Library/Fonts/AppleGothic.ttf'
fm.fontManager.addfont(_fp)
plt.rcParams['font.family'] = fm.FontProperties(fname=_fp).get_name()
plt.rcParams['axes.unicode_minus'] = False
sns.set_theme(style='whitegrid')

torch.manual_seed(42)
np.random.seed(42)

print(f'PyTorch version: {torch.__version__}')
device = torch.device('cpu')
print(f'Using device   : {device}')

---
## Part 1. 옵티마이저 (Optimizers)

### 1-1. 왜 다양한 옵티마이저가 필요한가?

기본 경사 하강법(SGD)은 단순하지만, 실제 손실 지형(loss landscape)에서는 비효율적입니다.

| 옵티마이저 | 업데이트 규칙 | 특징 |
|---|---|---|
| **SGD** | $w \leftarrow w - \eta \nabla L$ | 단순, 느림, 하이퍼파라미터 민감 |
| **Momentum** | $v \leftarrow \beta v + \nabla L$; $w \leftarrow w - \eta v$ | 관성으로 지그재그 감소 |
| **RMSProp** | $s \leftarrow \beta s + (1-\beta)g^2$; $w \leftarrow w - \frac{\eta}{\sqrt{s+\varepsilon}}g$ | 적응형 lr, 온라인 학습에 강함 |
| **Adam** | Momentum + RMSProp 결합, 편향 보정 포함 | 대부분 상황에서 잘 작동 |

### 1-2. 손실 지형 시각화

$f(x, y) = x^2 + 10y^2$ (길쭉한 이차 함수)에서 각 옵티마이저의 최솟값 탐색 경로를 비교합니다.

In [ ]:
# 최적화 궤적 시각화: f(x, y) = x² + 10y²
def f(x, y):     return x**2 + 10*y**2
def grad_f(x, y): return 2*x, 20*y

def run_sgd(x0, y0, lr, n):
    x, y = x0, y0; path = [(x, y)]
    for _ in range(n):
        gx, gy = grad_f(x, y); x -= lr*gx; y -= lr*gy; path.append((x, y))
    return path

def run_momentum(x0, y0, lr, beta, n):
    x, y, vx, vy = x0, y0, 0., 0.; path = [(x, y)]
    for _ in range(n):
        gx, gy = grad_f(x, y)
        vx = beta*vx + gx; vy = beta*vy + gy
        x -= lr*vx; y -= lr*vy; path.append((x, y))
    return path

def run_rmsprop(x0, y0, lr, beta, eps, n):
    x, y, sx, sy = x0, y0, 0., 0.; path = [(x, y)]
    for _ in range(n):
        gx, gy = grad_f(x, y)
        sx = beta*sx + (1-beta)*gx**2; sy = beta*sy + (1-beta)*gy**2
        x -= lr*gx/(np.sqrt(sx)+eps); y -= lr*gy/(np.sqrt(sy)+eps)
        path.append((x, y))
    return path

def run_adam(x0, y0, lr, b1, b2, eps, n):
    x, y, mx, my, vx, vy = x0, y0, 0., 0., 0., 0.; path = [(x, y)]
    for t in range(1, n+1):
        gx, gy = grad_f(x, y)
        mx = b1*mx+(1-b1)*gx;  my = b1*my+(1-b1)*gy
        vx = b2*vx+(1-b2)*gx**2; vy = b2*vy+(1-b2)*gy**2
        mxh = mx/(1-b1**t); myh = my/(1-b1**t)
        vxh = vx/(1-b2**t); vyh = vy/(1-b2**t)
        x -= lr*mxh/(np.sqrt(vxh)+eps); y -= lr*myh/(np.sqrt(vyh)+eps)
        path.append((x, y))
    return path

x0, y0, n_steps = 2.0, 2.0, 60
paths = {
    'SGD (lr=0.05)':        run_sgd(x0, y0, 0.05, n_steps),
    'Momentum (β=0.9)':     run_momentum(x0, y0, 0.05, 0.9, n_steps),
    'RMSProp':              run_rmsprop(x0, y0, 0.1, 0.9, 1e-8, n_steps),
    'Adam':                 run_adam(x0, y0, 0.3, 0.9, 0.999, 1e-8, n_steps),
}

xr = np.linspace(-2.5, 2.5, 300); yr = np.linspace(-2.5, 2.5, 300)
Xg, Yg = np.meshgrid(xr, yr); Zg = f(Xg, Yg)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))
colors = ['steelblue', 'tomato', 'seagreen', 'darkorange']
levels = [0.5, 2, 5, 10, 20, 40]

# 궤적 그림
for ax_i, (pair, title) in enumerate([
    ([0,1], 'SGD vs Momentum'),
    ([2,3], 'RMSProp vs Adam'),
]):
    ax = axes[ax_i]
    cs = ax.contour(Xg, Yg, Zg, levels=levels, cmap='Blues', alpha=0.6)
    ax.clabel(cs, fmt='%.0f', fontsize=8)
    ax.plot(0, 0, 'r*', ms=15, zorder=10, label='최솟값')
    for i in pair:
        name = list(paths.keys())[i]
        pts = paths[name]
        xs, ys = zip(*pts)
        ax.plot(xs, ys, 'o-', color=colors[i], ms=3, lw=1.5, label=name, alpha=0.85)
        ax.plot(xs[0], ys[0], 's', color=colors[i], ms=10, zorder=5)
    ax.set_title(title); ax.legend(fontsize=9)
    ax.set_xlabel('x'); ax.set_ylabel('y')

# 수렴 속도 비교 (로그 스케일)
ax = axes[2]
for (name, path), color in zip(paths.items(), colors):
    losses = [f(x, y) for x, y in path]
    ax.semilogy(losses, color=color, lw=2, label=name)
ax.set_title('수렴 속도 (로그 스케일)')
ax.set_xlabel('스텝'); ax.set_ylabel('f(x,y)'); ax.legend(fontsize=8)

plt.suptitle('최적화 궤적 비교: f(x,y) = x² + 10y²', fontsize=12)
plt.tight_layout(); plt.show()

print('=== 최종 손실 비교 ===')
for name, path in paths.items():
    print(f'  {name:<20}: f = {f(*path[-1]):.6f}')

In [ ]:
# Digits 데이터셋 준비
digits = load_digits()
X = digits.data.astype(np.float32)
y = digits.target

X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
X_tr_s = scaler.fit_transform(X_tr).astype(np.float32)
X_te_s = scaler.transform(X_te).astype(np.float32)

X_tr_t = torch.tensor(X_tr_s)
y_tr_t = torch.tensor(y_tr, dtype=torch.long)
X_te_t = torch.tensor(X_te_s)
y_te_t = torch.tensor(y_te, dtype=torch.long)

train_ds = TensorDataset(X_tr_t, y_tr_t)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)

print('=== Digits 데이터셋 ===')
print(f'  총 샘플: {len(X):,}개')
print(f'  특성 수: {X.shape[1]} (8×8 픽셀 → 64차원)')
print(f'  클래스: {list(digits.target_names)}  (10개)')
print(f'  Train  : {len(X_tr)}개  |  Test: {len(X_te)}개')

# 샘플 시각화
fig, axes = plt.subplots(2, 10, figsize=(14, 3))
for d in range(10):
    idx  = np.where(y == d)[0][0]
    idx2 = np.where(y == d)[0][1]
    axes[0, d].imshow(digits.images[idx],  cmap='gray_r'); axes[0, d].set_title(str(d), fontsize=10)
    axes[1, d].imshow(digits.images[idx2], cmap='gray_r')
    for r in range(2): axes[r, d].axis('off')
plt.suptitle('Digits 샘플 (클래스별 2개)', fontsize=11)
plt.tight_layout(); plt.show()

In [ ]:
# MLP 클래스 및 학습 함수 (이 셀은 이후 모든 실험에서 공유됩니다)

class MLP(nn.Module):
    """배치 정규화, 레이어 정규화, 드롭아웃을 지원하는 MLP"""

    def __init__(self, input_dim, hidden_dims, output_dim,
                 dropout=0.0, batch_norm=False, layer_norm=False):
        super().__init__()
        layers = []
        prev = input_dim
        for h in hidden_dims:
            layers.append(nn.Linear(prev, h))
            if batch_norm:
                layers.append(nn.BatchNorm1d(h))
            elif layer_norm:
                layers.append(nn.LayerNorm(h))
            layers.append(nn.ReLU())
            if dropout > 0:
                layers.append(nn.Dropout(dropout))
            prev = h
        layers.append(nn.Linear(prev, output_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)


def train_model(model, loader, X_val, y_val,
                n_epochs=100, lr=0.001, optimizer_type='adam',
                weight_decay=0.0, l1_lambda=0.0,
                scheduler_type=None, verbose=True):
    """
    optimizer_type: 'sgd', 'momentum', 'rmsprop', 'adam'
    scheduler_type: None, 'step', 'cosine', 'plateau', 'exp'
    """
    criterion = nn.CrossEntropyLoss()

    opt_map = {
        'sgd':      lambda: optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay),
        'momentum': lambda: optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay),
        'rmsprop':  lambda: optim.RMSprop(model.parameters(), lr=lr, weight_decay=weight_decay),
        'adam':     lambda: optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay),
    }
    optimizer = opt_map[optimizer_type]()

    scheduler = None
    if scheduler_type == 'step':
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)
    elif scheduler_type == 'cosine':
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=n_epochs)
    elif scheduler_type == 'plateau':
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5)
    elif scheduler_type == 'exp':
        scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

    train_losses, val_losses, train_accs, val_accs, lr_hist = [], [], [], [], []

    for epoch in range(n_epochs):
        model.train()
        ep_loss, correct, total = 0., 0, 0
        for Xb, yb in loader:
            optimizer.zero_grad()
            logits = model(Xb)
            loss = criterion(logits, yb)
            # L1 패널티 수동 추가
            if l1_lambda > 0:
                l1 = sum(p.abs().sum() for p in model.parameters())
                loss = loss + l1_lambda * l1
            loss.backward()
            optimizer.step()
            ep_loss  += loss.item() * len(Xb)
            correct  += (logits.argmax(1) == yb).sum().item()
            total    += len(yb)

        train_losses.append(ep_loss / total)
        train_accs.append(correct / total)
        lr_hist.append(optimizer.param_groups[0]['lr'])

        model.eval()
        with torch.no_grad():
            lv = model(X_val)
            val_loss = criterion(lv, y_val).item()
            val_acc  = (lv.argmax(1) == y_val).float().mean().item()
        val_losses.append(val_loss); val_accs.append(val_acc)

        if scheduler is not None:
            scheduler.step(val_loss) if scheduler_type == 'plateau' else scheduler.step()

        if verbose and (epoch + 1) % 25 == 0:
            print(f'Epoch {epoch+1:3d}/{n_epochs} | '
                  f'Train {train_losses[-1]:.4f}/{train_accs[-1]:.4f} | '
                  f'Val {val_loss:.4f}/{val_acc:.4f}')

    return train_losses, val_losses, train_accs, val_accs, lr_hist


# 모델 확인
demo = MLP(64, [128, 64], 10)
print('=== MLP 구조 (64 → 128 → 64 → 10) ===')
print(demo)
print(f'\n총 파라미터: {sum(p.numel() for p in demo.parameters()):,}')

In [ ]:
# 4가지 옵티마이저 비교 (Digits 10-class Classification)
opt_configs = [
    ('SGD (lr=0.01)',      'sgd',      0.01),
    ('Momentum (β=0.9)',   'momentum', 0.01),
    ('RMSProp (lr=0.001)', 'rmsprop',  0.001),
    ('Adam (lr=0.001)',    'adam',     0.001),
]
colors_opt = ['steelblue', 'tomato', 'seagreen', 'darkorange']

opt_results = {}
print('=== 옵티마이저 비교 (100 에포크) ===\n')
for name, opt_type, lr in opt_configs:
    torch.manual_seed(42)
    m = MLP(64, [128, 64], 10)
    t_l, v_l, t_a, v_a, _ = train_model(
        m, train_loader, X_te_t, y_te_t,
        n_epochs=100, lr=lr, optimizer_type=opt_type, verbose=False
    )
    opt_results[name] = (t_l, v_l, t_a, v_a)
    print(f'{name:<25}: Val Acc={v_a[-1]:.4f}  Val Loss={v_l[-1]:.4f}')

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
for (name, (_, v_l, _, v_a)), color in zip(opt_results.items(), colors_opt):
    axes[0].plot(v_l, color=color, lw=2, label=name)
    axes[1].plot(v_a, color=color, lw=2, label=f'{name} ({v_a[-1]:.4f})')

for ax, title, ylabel in [
    (axes[0], '옵티마이저별 Val Loss',     'CrossEntropy Loss'),
    (axes[1], '옵티마이저별 Val Accuracy', 'Accuracy'),
]:
    ax.set_title(title); ax.set_xlabel('에포크')
    ax.set_ylabel(ylabel); ax.legend(fontsize=9)
axes[1].set_ylim(0.5, 1.02)

plt.suptitle('옵티마이저 비교 (Digits)', fontsize=12)
plt.tight_layout(); plt.show()

---
## Part 2. 정규화 (Regularization)

### 2-1. 왜 정규화가 필요한가?

모델이 너무 크거나 학습 데이터가 적으면 **과적합(Overfitting)** 이 발생합니다.
정규화는 모델의 복잡도를 제한하여 **일반화(Generalization) 성능**을 높입니다.

### 2-2. 주요 정규화 기법

| 기법 | 설명 | PyTorch |
|---|---|---|
| **L2 (Weight Decay)** | $L_{\text{total}} = L + \lambda \sum w^2$ | `optimizer(..., weight_decay=λ)` |
| **L1** | $L_{\text{total}} = L + \lambda \sum |w|$ | 직접 구현 필요 |
| **Dropout** | 학습 시 뉴런을 확률 $p$로 무작위 비활성화 | `nn.Dropout(p)` |

**Dropout 핵심:**
- **학습 시**: 각 뉴런을 확률 $p$로 0으로 만들고, 남은 출력을 $\frac{1}{1-p}$로 스케일
- **추론 시**: 모든 뉴런을 사용 (`model.eval()` 호출 시 자동 처리)

In [ ]:
# 과적합 시나리오: 작은 학습 데이터 + 큰 모델
np.random.seed(42)
small_idx = np.random.choice(len(X_tr_t), 150, replace=False)
X_small_t = X_tr_t[small_idx]
y_small_t = y_tr_t[small_idx]

small_ds     = TensorDataset(X_small_t, y_small_t)
small_loader = DataLoader(small_ds, batch_size=32, shuffle=True)

torch.manual_seed(42)
model_overfit = MLP(64, [256, 256, 128], 10)
print(f'=== 과적합 시나리오: 150개 학습 샘플, 큰 모델 ===')
print(f'모델 파라미터: {sum(p.numel() for p in model_overfit.parameters()):,}개')

t_l_ov, v_l_ov, t_a_ov, v_a_ov, _ = train_model(
    model_overfit, small_loader, X_te_t, y_te_t,
    n_epochs=200, lr=0.001, verbose=False
)

fig, axes = plt.subplots(1, 2, figsize=(13, 4))
ep = range(1, 201)

axes[0].plot(ep, t_l_ov, 'b-', lw=2, label=f'Train Loss')
axes[0].plot(ep, v_l_ov, 'r-', lw=2, label=f'Val Loss')
axes[0].set_title('과적합: Loss 곡선'); axes[0].set_xlabel('에포크'); axes[0].set_ylabel('Loss')
axes[0].legend()

axes[1].plot(ep, t_a_ov, 'b-', lw=2, label=f'Train Acc ({t_a_ov[-1]:.4f})')
axes[1].plot(ep, v_a_ov, 'r-', lw=2, label=f'Val Acc   ({v_a_ov[-1]:.4f})')
axes[1].set_title('과적합: Accuracy 곡선'); axes[1].set_xlabel('에포크'); axes[1].set_ylabel('Accuracy')
axes[1].legend()

plt.suptitle('과적합(Overfitting): Train ↑, Val ↓ — 일반화 실패!', fontsize=12)
plt.tight_layout(); plt.show()

gap = t_a_ov[-1] - v_a_ov[-1]
print(f'\nTrain Acc  : {t_a_ov[-1]:.4f}')
print(f'Val   Acc  : {v_a_ov[-1]:.4f}')
print(f'일반화 Gap : {gap:.4f}  (클수록 과적합 심각)')

In [ ]:
# 정규화 기법 비교 (동일한 작은 데이터셋, 큰 모델)
reg_configs = [
    ('정규화 없음',        dict(dropout=0.0), 0.0,  0.0),
    ('L2 (wd=0.01)',      dict(dropout=0.0), 0.01, 0.0),
    ('L1 (λ=1e-4)',       dict(dropout=0.0), 0.0,  1e-4),
    ('Dropout (p=0.4)',   dict(dropout=0.4), 0.0,  0.0),
    ('L2 + Dropout',     dict(dropout=0.4), 0.01, 0.0),
]
colors_reg = ['gray', 'steelblue', 'purple', 'tomato', 'seagreen']

reg_results = {}
print('=== 정규화 기법 비교 ===\n')
print(f'{"기법":<20} {"Train Acc":>10} {"Val Acc":>10} {"Gap":>8}')
print('-' * 52)

for name, model_kw, wd, l1 in reg_configs:
    torch.manual_seed(42)
    m = MLP(64, [256, 256, 128], 10, **model_kw)
    t_l, v_l, t_a, v_a, _ = train_model(
        m, small_loader, X_te_t, y_te_t,
        n_epochs=200, lr=0.001, weight_decay=wd, l1_lambda=l1, verbose=False
    )
    reg_results[name] = (t_l, v_l, t_a, v_a)
    gap = t_a[-1] - v_a[-1]
    print(f'{name:<20} {t_a[-1]:>10.4f} {v_a[-1]:>10.4f} {gap:>8.4f}')

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
ep = range(1, 201)
for (name, (_, v_l, t_a, v_a)), color in zip(reg_results.items(), colors_reg):
    axes[0].plot(ep, v_l, color=color, lw=2, label=name)
    axes[1].plot(ep, v_a, color=color, lw=2, label=f'{name} ({v_a[-1]:.4f})')

for ax, title, ylabel in [
    (axes[0], 'Val Loss — 정규화 효과',     'Loss'),
    (axes[1], 'Val Accuracy — 정규화 효과', 'Accuracy'),
]:
    ax.set_title(title); ax.set_xlabel('에포크')
    ax.set_ylabel(ylabel); ax.legend(fontsize=8)

plt.suptitle('정규화 기법 비교 (과적합 환경, 150개 학습 샘플)', fontsize=12)
plt.tight_layout(); plt.show()

# Dropout 동작 확인 (train vs eval 모드)
model_test = MLP(64, [8], 10, dropout=0.5)
x_test = torch.randn(1, 64)
model_test.train()
out_train = model_test(x_test).detach()
model_test.eval()
out_eval  = model_test(x_test).detach()
print('\nDropout 동작 확인 (출력 최댓값):')
print(f'  train 모드 (Dropout 활성):  {out_train.max().item():.4f}')
print(f'  eval  모드 (Dropout 비활성): {out_eval.max().item():.4f}')

---
## Part 3. 배치 정규화 & 레이어 정규화

### 3-1. 배치 정규화 (Batch Normalization)

**내부 공변량 이동(Internal Covariate Shift)**: 학습 중 각 층의 입력 분포가 계속 변하는 현상.
이로 인해 학습 속도가 느려지고 높은 학습률 사용이 어렵습니다.

**BN 해결책:** 각 미니배치에서 정규화 수행

$$\hat{x} = \frac{x - \mu_B}{\sqrt{\sigma_B^2 + \varepsilon}}, \quad y = \gamma \hat{x} + \beta$$

| 항목 | 내용 |
|---|---|
| $\mu_B, \sigma_B^2$ | 미니배치의 평균과 분산 |
| $\gamma, \beta$ | 학습 가능한 스케일 & 시프트 파라미터 |
| **학습 시** | 미니배치 통계 사용 |
| **추론 시** | 학습 중 누적한 이동 평균 통계 사용 |

### 3-2. 레이어 정규화 (Layer Normalization)

BN은 배치 전체로 정규화하지만, **LayerNorm** 은 샘플 하나의 특성 차원으로 정규화합니다.
배치 크기가 작거나 RNN/Transformer에서 주로 사용됩니다.

In [ ]:
# BatchNorm vs No-BN: 수렴 속도 및 안정성 비교
bn_configs = [
    ('No BN  (lr=0.01)',  False, False, 0.01),
    ('No BN  (lr=0.001)', False, False, 0.001),
    ('BatchNorm (lr=0.01)',  True,  False, 0.01),
    ('BatchNorm (lr=0.001)', True,  False, 0.001),
]
colors_bn = ['steelblue', 'steelblue', 'tomato', 'tomato']
styles_bn  = ['--', '-', '--', '-']

bn_results = {}
print('=== BatchNorm 효과 비교 ===\n')
for name, use_bn, use_ln, lr in bn_configs:
    torch.manual_seed(42)
    m = MLP(64, [128, 64], 10, batch_norm=use_bn, layer_norm=use_ln)
    n_params = sum(p.numel() for p in m.parameters())
    t_l, v_l, t_a, v_a, _ = train_model(
        m, train_loader, X_te_t, y_te_t,
        n_epochs=100, lr=lr, verbose=False
    )
    bn_results[name] = (v_l, v_a)
    print(f'{name:<26}  params={n_params:,}  Val Acc={v_a[-1]:.4f}')

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
for (name, (v_l, v_a)), color, ls in zip(bn_results.items(), colors_bn, styles_bn):
    label_acc = f'{name} ({v_a[-1]:.4f})'
    axes[0].plot(v_l, color=color, lw=2, ls=ls, label=name)
    axes[1].plot(v_a, color=color, lw=2, ls=ls, label=label_acc)

for ax, title, ylabel in [
    (axes[0], 'BatchNorm 효과: Val Loss',     'Loss'),
    (axes[1], 'BatchNorm 효과: Val Accuracy', 'Accuracy'),
]:
    ax.set_title(title); ax.set_xlabel('에포크')
    ax.set_ylabel(ylabel); ax.legend(fontsize=9)

plt.suptitle('Batch Normalization: 더 높은 lr에서도 안정적 학습!', fontsize=12)
plt.tight_layout(); plt.show()

In [ ]:
# BN vs LayerNorm vs No-Norm 비교
norm_configs = [
    ('No Norm',    False, False),
    ('BatchNorm',  True,  False),
    ('LayerNorm',  False, True),
]
colors_norm = ['steelblue', 'tomato', 'seagreen']

norm_results = {}
print('=== BN vs LayerNorm vs No-Norm (lr=0.01) ===\n')
for name, bn, ln in norm_configs:
    torch.manual_seed(42)
    m = MLP(64, [128, 64], 10, batch_norm=bn, layer_norm=ln)
    t_l, v_l, t_a, v_a, _ = train_model(
        m, train_loader, X_te_t, y_te_t,
        n_epochs=100, lr=0.01, verbose=False
    )
    norm_results[name] = (v_l, v_a)
    print(f'{name:<12}: Val Acc={v_a[-1]:.4f}  최고 Val Acc={max(v_a):.4f}')

fig, axes = plt.subplots(1, 2, figsize=(13, 4))
for (name, (v_l, v_a)), color in zip(norm_results.items(), colors_norm):
    axes[0].plot(v_l, color=color, lw=2, label=name)
    axes[1].plot(v_a, color=color, lw=2, label=f'{name} ({v_a[-1]:.4f})')

for ax, title, ylabel in [
    (axes[0], '정규화별 Val Loss',     'Loss'),
    (axes[1], '정규화별 Val Accuracy', 'Accuracy'),
]:
    ax.set_title(title); ax.set_xlabel('에포크')
    ax.set_ylabel(ylabel); ax.legend(fontsize=9)

plt.suptitle('No Norm vs BatchNorm vs LayerNorm (lr=0.01)', fontsize=12)
plt.tight_layout(); plt.show()

print('\n[참고]')
print('  BatchNorm: 배치 차원으로 정규화 → CNN, MLP에 적합')
print('  LayerNorm: 특성 차원으로 정규화 → RNN, Transformer에 적합')

---
## Part 4. 학습률 스케줄링 (Learning Rate Scheduling)

### 왜 학습률을 조정하는가?

- **초기 학습**: 큰 학습률 → 빠른 수렴
- **후반 학습**: 작은 학습률 → 세밀한 최적화, 진동 방지

| 스케줄러 | 방식 | 특징 |
|---|---|---|
| **StepLR** | N 에포크마다 lr × γ | 단순, 예측 가능 |
| **CosineAnnealingLR** | 코사인 함수로 lr 감소 | 부드러운 감소, warm restart 응용 |
| **ExponentialLR** | 매 에포크마다 lr × γ | 지수적 감소 |
| **ReduceLROnPlateau** | Val Loss가 개선되지 않을 때 lr 감소 | 적응형, 실전에서 많이 사용 |

```python
# 사용 예시
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)
for epoch in range(n_epochs):
    # ... 학습 루프 ...
    scheduler.step()          # 에포크 끝에 호출
    # ReduceLROnPlateau는: scheduler.step(val_loss)
```

In [ ]:
# 학습률 스케줄러 시각화 (실제 학습 없이 lr 변화만 확인)
n_epochs_vis = 100
init_lr = 0.01

dummy_p = nn.Parameter(torch.tensor(0.0))  # 더미 파라미터

schedulers_vis = {
    '고정 lr':            (None, {}),
    'StepLR (s=30, γ=0.5)': ('step',   {'step_size': 30, 'gamma': 0.5}),
    'CosineAnnealingLR':   ('cosine', {'T_max': n_epochs_vis}),
    'ExponentialLR (γ=0.95)': ('exp',  {'gamma': 0.95}),
}

fig, ax = plt.subplots(figsize=(10, 4))
colors_sch = ['gray', 'steelblue', 'tomato', 'seagreen']

for (name, (sch_type, sch_kw)), color in zip(schedulers_vis.items(), colors_sch):
    opt_v = optim.SGD([dummy_p], lr=init_lr)
    if sch_type == 'step':
        sch = torch.optim.lr_scheduler.StepLR(opt_v, **sch_kw)
    elif sch_type == 'cosine':
        sch = torch.optim.lr_scheduler.CosineAnnealingLR(opt_v, **sch_kw)
    elif sch_type == 'exp':
        sch = torch.optim.lr_scheduler.ExponentialLR(opt_v, **sch_kw)
    else:
        sch = None
    lrs = []
    for ep in range(n_epochs_vis):
        lrs.append(opt_v.param_groups[0]['lr'])
        if sch: sch.step()
    ax.plot(range(1, n_epochs_vis+1), lrs, color=color, lw=2, label=name)

ax.set_title(f'학습률 스케줄러별 lr 변화 (초기 lr={init_lr})')
ax.set_xlabel('에포크'); ax.set_ylabel('Learning Rate')
ax.legend()
plt.tight_layout(); plt.show()

print('=== 학습률 핵심 포인트 ===')
print('  - 너무 크면: 발산(loss 폭발)')
print('  - 너무 작으면: 수렴 매우 느림')
print('  - 스케줄링: 초반 크게 → 후반 작게 = 속도 + 정밀도')

In [ ]:
# 스케줄러 학습 비교
sch_configs = [
    ('Adam (고정 lr=0.001)',    'adam', None,     0.001),
    ('Adam + StepLR',           'adam', 'step',   0.01),
    ('Adam + CosineAnneal',     'adam', 'cosine', 0.01),
    ('Adam + ExponentialLR',    'adam', 'exp',    0.01),
]
colors_sch2 = ['gray', 'steelblue', 'tomato', 'seagreen']

sch_results = {}
print('=== 학습률 스케줄링 비교 (100 에포크) ===\n')
for name, opt_type, sch_type, lr in sch_configs:
    torch.manual_seed(42)
    m = MLP(64, [128, 64], 10)
    t_l, v_l, t_a, v_a, lr_hist = train_model(
        m, train_loader, X_te_t, y_te_t,
        n_epochs=100, lr=lr, optimizer_type=opt_type,
        scheduler_type=sch_type, verbose=False
    )
    sch_results[name] = (v_a, lr_hist)
    print(f'{name:<30}: 최종 Val Acc={v_a[-1]:.4f}  최고 Val Acc={max(v_a):.4f}')

fig, axes = plt.subplots(1, 2, figsize=(13, 4))
for (name, (v_a, lr_hist)), color in zip(sch_results.items(), colors_sch2):
    axes[0].plot(range(1, 101), v_a,     color=color, lw=2, label=f'{name} ({v_a[-1]:.4f})')
    axes[1].plot(range(1, 101), lr_hist, color=color, lw=2, label=name)

axes[0].set_title('스케줄러별 Val Accuracy')
axes[0].set_xlabel('에포크'); axes[0].set_ylabel('Accuracy')
axes[0].legend(fontsize=8)

axes[1].set_title('스케줄러별 lr 변화')
axes[1].set_xlabel('에포크'); axes[1].set_ylabel('Learning Rate')
axes[1].legend(fontsize=8)

plt.suptitle('학습률 스케줄링 비교', fontsize=12)
plt.tight_layout(); plt.show()

---
## Exercise

### Exercise 1. 정규화 전략 심층 비교

전체 학습 데이터(X_tr_t, y_tr_t)를 사용하여 아래 4가지 설정을 비교하세요.

| 설정 | 내용 |
|---|---|
| 기준 | No regularization (dropout=0, weight_decay=0) |
| A | Dropout p=0.3 |
| B | L2 weight_decay=0.001 |
| C | Dropout p=0.3 + L2 weight_decay=0.001 |

**요구사항:**
- 모델 구조: `hidden_dims=[128, 64]`, `n_epochs=100`, Adam, `lr=0.001`
- Val Accuracy 학습 곡선 시각화 (4개 선)
- 최종 Val Accuracy 및 Train Accuracy 출력 (일반화 Gap 포함)

In [ ]:
# Exercise 1: 정규화 전략 비교
configs_ex1 = {
    '기준 (No Reg)':             dict(dropout=0.0, weight_decay=0.0),
    'A: Dropout p=0.3':          dict(dropout=0.3, weight_decay=0.0),
    'B: L2 wd=0.001':            dict(dropout=0.0, weight_decay=0.001),
    'C: Dropout + L2':           dict(dropout=0.3, weight_decay=0.001),
}

ex1_results = {}

for name, cfg in configs_ex1.items():
    # Your code here: MLP 생성, train_model 호출, 결과 저장
    pass

# Your code here: Val Accuracy 학습 곡선 시각화

# Your code here: 최종 Val Acc, Train Acc, Gap 출력

### Exercise 2. BatchNorm + Dropout 조합 최적화

아래 4가지 조합에서 최적의 설정을 찾으세요.

| 모델 | batch_norm | dropout |
|---|---|---|
| Vanilla | False | 0.0 |
| BN only | True | 0.0 |
| Dropout only | False | 0.3 |
| BN + Dropout | True | 0.3 |

**요구사항:**
- 구조: `hidden_dims=[256, 128, 64]`, `n_epochs=100`, Adam, `lr=0.01`
- Val Loss + Val Accuracy 학습 곡선 시각화
- 최종 성능 표 출력 (Val Acc, 파라미터 수 포함)

**힌트:** BatchNorm 레이어도 파라미터를 추가합니다!

In [ ]:
# Exercise 2: BN + Dropout 조합
bn_drop_configs = {
    'Vanilla':      dict(batch_norm=False, dropout=0.0),
    'BN only':      dict(batch_norm=True,  dropout=0.0),
    'Dropout only': dict(batch_norm=False, dropout=0.3),
    'BN + Dropout': dict(batch_norm=True,  dropout=0.3),
}

ex2_results = {}

for name, cfg in bn_drop_configs.items():
    # Your code here: MLP 생성 (hidden_dims=[256, 128, 64], output_dim=10, lr=0.01)

    # Your code here: train_model 호출, 결과 저장
    pass

# Your code here: Val Loss, Val Accuracy 곡선 시각화

# Your code here: 최종 성능 표 (Val Acc, 파라미터 수)

### Exercise 3. (도전) ReduceLROnPlateau + Early Stopping

**Early Stopping**: 검증 손실이 일정 에포크 동안 개선되지 않으면 학습을 조기 종료합니다.

```
최고 val_loss 갱신 시 → 모델 저장, 카운터 초기화
개선 없을 때         → 카운터 +1
카운터 ≥ patience   → 학습 종료
```

**요구사항:**
1. `train_with_early_stopping()` 함수 구현
   - `ReduceLROnPlateau` 스케줄러 사용 (patience=5, factor=0.5)
   - Early stopping: patience=15
2. 모델: `MLP(64, [128, 64], 10)`, Adam, 초기 `lr=0.01`, `max_epochs=300`
3. 결과 출력: 실제 학습 에포크 수, 최종 Val Acc, lr 변화 곡선

**힌트:**
```python
import copy
best_model_state = copy.deepcopy(model.state_dict())  # 모델 상태 저장
model.load_state_dict(best_model_state)                # 최고 상태로 복원
```

In [ ]:
import copy

def train_with_early_stopping(model, loader, X_val, y_val,
                               max_epochs=300, lr=0.01, patience=15):
    """
    ReduceLROnPlateau + Early Stopping 구현
    Returns: val_accs, lr_history, actual_epochs
    """
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Your code here: ReduceLROnPlateau 스케줄러 생성 (patience=5, factor=0.5)

    best_val_loss = float('inf')
    patience_counter = 0
    best_state = None

    val_accs  = []
    lr_history = []

    for epoch in range(max_epochs):
        # Your code here: 학습 루프 (train_loader 사용)

        # Your code here: 검증 (val_loss, val_acc 계산)

        # Your code here: val_accs, lr_history 기록

        # Your code here: scheduler.step(val_loss)

        # Your code here: Early stopping 체크
        #   - val_loss < best_val_loss → best 갱신, 카운터 초기화, 모델 저장
        #   - 그렇지 않으면 카운터 증가 → patience 초과 시 break
        pass

    # Your code here: 최고 모델 복원

    return val_accs, lr_history, epoch + 1


# Your code here: 실험 실행 및 결과 시각화
torch.manual_seed(42)
model_es = MLP(64, [128, 64], 10)

# val_accs, lr_history, actual_ep = train_with_early_stopping(
#     model_es, train_loader, X_te_t, y_te_t, max_epochs=300, lr=0.01, patience=15
# )
# print(f'실제 학습 에포크: {actual_ep} / 300')
# print(f'최종 Val Accuracy: {val_accs[-1]:.4f}')

---
## Summary

| 개념 | 핵심 내용 |
|---|---|
| **SGD** | 기본 경사 하강법, 학습률에 민감, 지그재그 수렴 |
| **Momentum** | 관성 누적으로 지그재그 감소, 빠른 수렴 |
| **RMSProp** | 방향별 적응적 학습률, 비정상(non-stationary) 데이터에 강함 |
| **Adam** | Momentum + RMSProp, 대부분 상황에서 첫 번째 선택 |
| **L1 규제** | $\sum|w|$ 패널티, 희소(sparse) 가중치 유도 |
| **L2 규제** | $\sum w^2$ 패널티, 가중치를 0 근처로 수축 |
| **Dropout** | 학습 시 무작위 뉴런 비활성, 앙상블 효과 |
| **BatchNorm** | 미니배치 정규화, 학습 안정성↑, 높은 lr 허용 |
| **LayerNorm** | 샘플별 특성 정규화, RNN/Transformer에 적합 |
| **StepLR** | N 에포크마다 lr × γ, 예측 가능한 감소 |
| **CosineAnnealingLR** | 코사인 곡선으로 부드러운 감소 |
| **ReduceLROnPlateau** | Val Loss 정체 시 자동 감소, 실전에서 많이 사용 |
| **Early Stopping** | Val Loss 개선 없으면 조기 종료, 과적합 방지 |

---

**다음 강의 (Week 12):** CNN — 합성곱 신경망, 풀링, 이미지 분류